In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Network

In [5]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
        

In [6]:
net = LeNet()

# Train

In [ ]:
import torch.optim as optim

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.MSELoss()

epoch = 10
for i in range(epoch):
    optimizer.zero_grad()
    
    output = net(input)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step() 